## Advanced options for multioutput handling

### Imports

In [1]:
import os
# set the device to run
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

os.makedirs('data', exist_ok=True)

import pandas as pd
from pandas import Series, DataFrame

from sklearn.model_selection import train_test_split

from py_boost import GradientBoosting 

# strategies to deal with multiple outputs
from py_boost.multioutput.sketching import *
from py_boost.multioutput.target_splitter import *

### Download data from openml

In [2]:
!wget https://www.openml.org/data/get_csv/19335692/file1c556677f875.csv -O data/helena.csv

--2021-11-11 03:10:09--  https://www.openml.org/data/get_csv/19335692/file1c556677f875.csv
Resolving www.openml.org (www.openml.org)... 131.155.11.11
Connecting to www.openml.org (www.openml.org)|131.155.11.11|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/plain]
Saving to: ‘data/helena.csv’

data/helena.csv         [        <=>         ]  14.56M  9.84MB/s    in 1.5s    

2021-11-11 03:10:11 (9.84 MB/s) - ‘data/helena.csv’ saved [15271704]



In [3]:
data = pd.read_csv('data/helena.csv')

data

,class,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27
0,41,0.005521,0.080556,0.110417,0.490822,0.586406,0.066414,0.092206,0.116352,0.379310,...,-0.342986,78.6894,17.237800,21.504200,14.43730,17.378000,9.61674,-0.609370,1.044830,1.481790
1,92,0.049398,0.147917,0.541667,0.542865,0.515608,0.105128,0.475550,0.049555,0.383460,...,2.639370,59.7879,5.393410,3.819610,11.49240,3.929470,5.91423,1.409210,4.749540,1.103820
2,24,0.548663,1.000000,1.000000,0.397029,0.627398,1.023440,1.004220,0.027381,0.451337,...,0.137427,58.1429,-3.365980,-0.037489,10.63470,2.660180,3.93377,-0.898220,2.137790,1.054470
3,29,0.023073,0.206250,0.238889,0.622998,0.764067,0.202599,0.177892,0.071232,0.531712,...,0.477009,55.4798,-1.051090,-4.755360,13.36710,2.852060,9.65162,0.224397,-0.220216,-0.273287
4,91,0.224427,0.433333,0.902083,0.814199,0.576879,0.344413,0.822975,0.026121,0.425875,...,0.521306,76.8475,-19.371700,32.270700,9.41442,4.343450,8.67710,-1.587580,1.117870,-0.545338
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65191,88,0.007292,0.152083,0.061111,0.114431,0.406104,0.143170,0.053086,0.129365,0.215442,...,1.265300,53.2951,-1.416430,2.173900,13.66950,1.588520,2.02855,0.619052,0.622377,-0.363035
65192,77,0.411279,1.000000,0.430556,0.503805,0.207163,1.003740,0.412067,0.017673,0.044771,...,-2.842380,91.1178,-0.009207,-2.224830,1.30504,0.898489,1.80362,-2.726140,-0.184329,-0.476441
65193,71,0.999352,1.000000,1.000000,0.501360,0.501075,0.999384,0.999414,0.009636,0.000648,...,0.213472,84.4141,2.042450,13.849800,7.24428,1.443890,4.00495,-0.749115,1.025130,0.096257
65194,24,0.206175,0.383333,0.944444,0.749915,0.550936,0.292477,0.830921,0.033542,0.430515,...,0.879472,61.4110,17.354200,5.566660,16.22600,10.049400,6.04195,0.400956,0.375599,0.644575


In [4]:
data['class'].value_counts()

78    4005
55    3063
40    2992
39    2623
38    2216
      ... 
56     121
75     121
32     119
34     116
10     111
Name: class, Length: 100, dtype: int64

#### Looks like we have 100 classes here ...

In [5]:
X = data.drop('class', axis=1).values.astype('float32')
y = data['class'].values.astype('int32')

X, X_test, y, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

### Train multiclass model

Simple usecase for training multiclass is the same as for regression. By default py_boost builds multioutout trees to handle multioutput problems, single tree outputs a vector of length 100 for 100 class task


In [6]:
%%time
model = GradientBoosting('crossentropy',
                         ntrees=10000, lr=0.03, verbose=100, es=300, lambda_l2=1,
                         subsample=1, colsample=1, min_data_in_leaf=10,
                         max_bin=256, max_depth=6)

model.fit(X, y, 
          eval_sets = [{'X': X_test, 'y': y_test}])

[03:10:13] Stdout logging level is INFO.
[03:10:13] GDBT train starts. Max iter 10000, early stopping rounds 300
[03:10:13] Iter 0; Sample 0, Crossentropy = 4.3048272133052015; 
[03:10:17] Iter 100; Sample 0, Crossentropy = 2.861118313424075; 
[03:10:21] Iter 200; Sample 0, Crossentropy = 2.7282271821664423; 
[03:10:26] Iter 300; Sample 0, Crossentropy = 2.6778634096094738; 
[03:10:30] Iter 400; Sample 0, Crossentropy = 2.650289255702367; 
[03:10:35] Iter 500; Sample 0, Crossentropy = 2.6342463294280942; 
[03:10:40] Iter 600; Sample 0, Crossentropy = 2.6234859605890155; 
[03:10:44] Iter 700; Sample 0, Crossentropy = 2.615059676634403; 
[03:10:49] Iter 800; Sample 0, Crossentropy = 2.6071631379546325; 
[03:10:53] Iter 900; Sample 0, Crossentropy = 2.600936181754179; 
[03:10:58] Iter 1000; Sample 0, Crossentropy = 2.5963903735180125; 
[03:11:03] Iter 1100; Sample 0, Crossentropy = 2.5918323999508037; 
[03:11:07] Iter 1200; Sample 0, Crossentropy = 2.588358199849226; 
[03:11:12] Iter 1300

In [7]:
%%time
pred = model.predict(X_test)

pred.shape

CPU times: user 1.96 s, sys: 736 ms, total: 2.69 s
Wall time: 2.69 s


(13040, 100)

### Sketching strategies to speedup training

Training multioutput models is more time consuming than single output. That's why we implement few strategies to simplify tree structure search via gradinet matrix sketching:

* ***RandomSamplingSketch*** (recommended)
* ***TopOutputsSketch***
* ***SVDSketch*** (needs RAPIDS (cuml) to be installed)

Let's check, how it works

In [8]:
%%time

sketch = RandomSamplingSketch(10)

model = GradientBoosting('crossentropy',
                         ntrees=10000, lr=0.03, verbose=100, es=300, lambda_l2=1,
                         subsample=1, colsample=1, min_data_in_leaf=10,
                         max_bin=256, max_depth=6,
                         multioutput_sketch=sketch
                        )

model.fit(X, y, eval_sets = [{'X': X_test, 'y': y_test}])

[03:12:15] Stdout logging level is INFO.
[03:12:15] GDBT train starts. Max iter 10000, early stopping rounds 300
[03:12:15] Iter 0; Sample 0, Crossentropy = 4.36672978708813; 
[03:12:16] Iter 100; Sample 0, Crossentropy = 2.900780767991263; 
[03:12:18] Iter 200; Sample 0, Crossentropy = 2.7606439456349694; 
[03:12:19] Iter 300; Sample 0, Crossentropy = 2.700749634336184; 
[03:12:20] Iter 400; Sample 0, Crossentropy = 2.6669122496840765; 
[03:12:22] Iter 500; Sample 0, Crossentropy = 2.643409104312327; 
[03:12:23] Iter 600; Sample 0, Crossentropy = 2.6286258707153114; 
[03:12:25] Iter 700; Sample 0, Crossentropy = 2.6165776897853523; 
[03:12:26] Iter 800; Sample 0, Crossentropy = 2.60684161177947; 
[03:12:27] Iter 900; Sample 0, Crossentropy = 2.5987716251359774; 
[03:12:29] Iter 1000; Sample 0, Crossentropy = 2.5918193315402154; 
[03:12:30] Iter 1100; Sample 0, Crossentropy = 2.587307146141814; 
[03:12:32] Iter 1200; Sample 0, Crossentropy = 2.5825257239125805; 
[03:12:33] Iter 1300; S

In [9]:
pred = model.predict(X_test)
pred.shape

(13040, 100)

#### Nice speed up and some times even better accuracy !

#### Its even faster than catboost with close to the same setup :)

In [10]:
%%time
from catboost import CatBoostClassifier

model = CatBoostClassifier(
                       grow_policy='Depthwise', 
                       bootstrap_type='Bernoulli',
                       subsample=1.,
                       border_count=256, 
                       iterations=10000, 
                       od_wait=300,
                       max_depth=6, 
                       devices='0:0', 
                       learning_rate=0.03, 
                       l2_leaf_reg=1, 
                       min_data_in_leaf=10, 
                       score_function='L2',
                       model_shrink_mode='Constant',
                       **{'task_type': 'GPU', 'verbose': 100, }
                    )

model.fit(X, y, eval_set = (X_test, y_test))    

0:	learn: 4.2940499	test: 4.3045904	best: 4.3045904 (0)	total: 41.1ms	remaining: 6m 51s
100:	learn: 2.7026067	test: 2.8639822	best: 2.8639822 (100)	total: 3.81s	remaining: 6m 13s
200:	learn: 2.4526127	test: 2.7287553	best: 2.7287553 (200)	total: 7.35s	remaining: 5m 58s
300:	learn: 2.3213711	test: 2.6789287	best: 2.6789287 (300)	total: 10.8s	remaining: 5m 46s
400:	learn: 2.2232473	test: 2.6516266	best: 2.6516266 (400)	total: 14.1s	remaining: 5m 38s
500:	learn: 2.1449559	test: 2.6351176	best: 2.6351176 (500)	total: 17.4s	remaining: 5m 29s
600:	learn: 2.0773085	test: 2.6237371	best: 2.6237371 (600)	total: 20.6s	remaining: 5m 21s
700:	learn: 2.0146058	test: 2.6149348	best: 2.6149318 (699)	total: 23.7s	remaining: 5m 15s
800:	learn: 1.9531144	test: 2.6072940	best: 2.6072838 (799)	total: 26.9s	remaining: 5m 9s
900:	learn: 1.8969737	test: 2.6016197	best: 2.6016197 (900)	total: 30.1s	remaining: 5m 4s
1000:	learn: 1.8410818	test: 2.5963367	best: 2.5963367 (1000)	total: 33.3s	remaining: 4m 59s
11

### Target splitting strategies

As it was mentioned above, py_boost builds a single tree for all outputs. Commonly its recommended strategy, but there are some options. Here are possible strategies:

* ***'Single'*** stands for default strategy
* ***'OneVsAll'*** (not recommended) stands for building separate trees for the each output
* ***RandomGroupsSplitter*** (experimental) randomly groups the outputs at the each iteration and build a tree per group. For example, you can build 3 trees predicting 100 classes
* ***Custom*** build your own strategy to split outputs to the groups

#### Now, let's try to check RandomGroupsSplitter together with the sketching 

In [11]:
%%time

splitter = RandomGroupsSplitter(3)
sketch = RandomSamplingSketch(10)

model = GradientBoosting('crossentropy',
                         ntrees=10000, lr=0.03, verbose=100, es=300, lambda_l2=1,
                         subsample=1, colsample=1, min_data_in_leaf=10,
                         max_bin=256, max_depth=6,
                         multioutput_sketch=sketch,
                         target_splitter=splitter
                        )

model.fit(X, y, eval_sets = [{'X': X_test, 'y': y_test}])

[03:14:31] Stdout logging level is INFO.
[03:14:31] GDBT train starts. Max iter 10000, early stopping rounds 300
[03:14:31] Iter 0; Sample 0, Crossentropy = 4.32698038298462; 
[03:14:35] Iter 100; Sample 0, Crossentropy = 2.8532840522403506; 
[03:14:39] Iter 200; Sample 0, Crossentropy = 2.727036329549456; 
[03:14:43] Iter 300; Sample 0, Crossentropy = 2.6746698540305776; 
[03:14:47] Iter 400; Sample 0, Crossentropy = 2.646433247863518; 
[03:14:51] Iter 500; Sample 0, Crossentropy = 2.6275252474157638; 
[03:14:55] Iter 600; Sample 0, Crossentropy = 2.614278060267745; 
[03:14:59] Iter 700; Sample 0, Crossentropy = 2.60369742806435; 
[03:15:03] Iter 800; Sample 0, Crossentropy = 2.5962074316917985; 
[03:15:06] Iter 900; Sample 0, Crossentropy = 2.589461930646581; 
[03:15:10] Iter 1000; Sample 0, Crossentropy = 2.58452233957608; 
[03:15:14] Iter 1100; Sample 0, Crossentropy = 2.580134470134513; 
[03:15:18] Iter 1200; Sample 0, Crossentropy = 2.575955113323579; 
[03:15:22] Iter 1300; Sampl

#### Didn't improve the score, but at least we could try :)